En esta actividad se utulizarán métodos de selección de características: selección hacia adelante y eliminación hacia atrás. 

Trabajaremos con los datos del Wine Quality Data Set, generados por Paulo Cortez y un grupo de investigación de la University of Minho, en Portugal, disponibles en [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/186/wine+quality). 

Los investigadores utilizaron estos datos para tratar de modelar las preferencias de consumo de vino, generando un índice de calidad. Ahora, trataremos de mejorar la precisión del modelo a través de metodologías de selección de características. 

Primero importaremos los datos

In [3]:
import pandas as pd
df = pd.read_csv("A1.4 Vino Tinto.csv")
print(df.shape)
print(df.head(5))

(1599, 12)
   acidezFija  acidezVolatil  acidoCitrico  azucarResidual  cloruros  \
0         7.4           0.70          0.00             1.9     0.076   
1         7.8           0.88          0.00             2.6     0.098   
2         7.8           0.76          0.04             2.3     0.092   
3        11.2           0.28          0.56             1.9     0.075   
4         7.4           0.70          0.00             1.9     0.076   

   dioxidoAzufreLibre  dioxidoAzufreTotal  densidad    pH  sulfatos  alcohol  \
0                11.0                34.0    0.9978  3.51      0.56      9.4   
1                25.0                67.0    0.9968  3.20      0.68      9.8   
2                15.0                54.0    0.9970  3.26      0.65      9.8   
3                17.0                60.0    0.9980  3.16      0.58      9.8   
4                11.0                34.0    0.9978  3.51      0.56      9.4   

   calidad  
0        5  
1        5  
2        5  
3        6  
4        5

Ahora dividiremos nuestros datos en datos de entrenamiento y datos de prueba

In [7]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, train_size = 0.8)

print(train.shape)
print(test.shape)
if df.shape[0] == (train.shape[0] + test.shape[0]):
    print("La cantidad de observaciones en las particiones son iguales a la cantidad original.")


(1279, 12)
(320, 12)
La cantidad de observaciones en las particiones son iguales a la cantidad original.


Ahora procederemos con los métodos de selección.

In [55]:
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression
import numpy as np

x_train = train.drop(columns=['calidad'])
y_train = train['calidad']
x_test  = test.drop(columns=['calidad'])
y_test  = test['calidad']


SFS = SequentialFeatureSelector(estimator = LinearRegression(), k_features = (2,8), forward = True, scoring = 'r2', cv = 10)
SFS.fit(x_train,y_train)

print("Variables seleccionadas:",SFS.k_feature_names_)
print(SFS.k_score_,"\n")
#print("\n",SFS.subsets_)

for i in SFS.subsets_.items():
    print("Variables seleccionadas:",i[0],i[1]['feature_names'])


Variables seleccionadas: ('acidezVolatil', 'cloruros', 'dioxidoAzufreLibre', 'dioxidoAzufreTotal', 'pH', 'sulfatos', 'alcohol')
0.3402364475745765 

Variables seleccionadas: 1 ('alcohol',)
Variables seleccionadas: 2 ('acidezVolatil', 'alcohol')
Variables seleccionadas: 3 ('acidezVolatil', 'sulfatos', 'alcohol')
Variables seleccionadas: 4 ('acidezVolatil', 'cloruros', 'sulfatos', 'alcohol')
Variables seleccionadas: 5 ('acidezVolatil', 'cloruros', 'dioxidoAzufreTotal', 'sulfatos', 'alcohol')
Variables seleccionadas: 6 ('acidezVolatil', 'cloruros', 'dioxidoAzufreTotal', 'pH', 'sulfatos', 'alcohol')
Variables seleccionadas: 7 ('acidezVolatil', 'cloruros', 'dioxidoAzufreLibre', 'dioxidoAzufreTotal', 'pH', 'sulfatos', 'alcohol')
Variables seleccionadas: 8 ('acidezVolatil', 'azucarResidual', 'cloruros', 'dioxidoAzufreLibre', 'dioxidoAzufreTotal', 'pH', 'sulfatos', 'alcohol')


Aquí podemos observar los resultados de la selección hacia adelante, mostrando qué variables parecen ser las de mayor relevancia a la hora de hacer el modelo

In [67]:
from sklearn.metrics import r2_score
columnas = list(SFS.k_feature_names_)
lr_adelante = LinearRegression().fit(x_train[columnas], y_train)
y_pred = lr_adelante.predict(x_test[columnas])

r2_test = r2_score(y_test, y_pred)
print(f"R^2: {r2_test:.4f}")
# 

R^2: 0.3457


Aquí podemos ver ka capacidad de medición del modelo medida mediante R^2 con un valor de 0.3457. 

Ahora procederemos a aplicar el método de selección hacia atrás.

In [62]:
x_train_seleccionadas = x_train[columnas]
x_test_seleccionadas = x_test[columnas]

SFS_atras = SequentialFeatureSelector(estimator = LinearRegression(), k_features = (2,5), forward = False, scoring = 'r2', cv = 10)
SFS_atras.fit(x_train_seleccionadas,y_train)

print("Variables seleccionadas:",SFS_atras.k_feature_names_)
print(SFS_atras.k_score_,"\n")
#print("\n",SFS.subsets_)


Variables seleccionadas: ('acidezVolatil', 'cloruros', 'dioxidoAzufreTotal', 'sulfatos', 'alcohol')
0.3359861328870746 



Una vez obtenidas las variables seleccionadas después de aplicar el método de selección hacia atrás sobre los resultados de haber aplicado el método de selección hacia adelante, volveremos a calcular el R^2 para comparar la capacidad de medición de ambos modelos

In [69]:
columnas_atras = list(SFS_atras.k_feature_names_)
lr_final = LinearRegression().fit(x_train_seleccionadas[columnas_atras], y_train)
y_pred_atras = lr_final.predict(x_test_seleccionadas[columnas_atras])

r2_test_atras = r2_score(y_test, y_pred_atras)
Eprint(f"R^2: {r2_test_atras:.4f}")

R^2: 0.3352


Al observar y comparar ambos resultados obtenidos de R^2 para ambos modelos podemos notar lo siguiente:

- El primer modelo obtenido tiene una R^2 de 0.3457 mientras que el segundo tiene una de 0.3352, habiendo una diferencia de 0.0105 en sus valores.
- El primer modelo hace uso de 7 variables, mientras que el segundo hace uso de 5 variables.

Debido a esto, considero que el requerir 2 variables menos a costa de un 0.0105 de capacidad de medición termina siendo una diferencia no tan significativa en este caso, por lo que si buscamos agilizar procesos a la hora de utilizar alguno de estos modelos, considero que el más adecuado sería el segundo modelo, haciendo uso de las variables de alcohol, acidéz volatil, sulfatos, cloruros y dióxido de azufre total, mientras que descarta el resto de variables seleccionadas anteriormente como azucares y nivel de pH.